In [1]:
import pickle
import numpy as np
import pandas as pd
import centrex_TlF as centrex
import matplotlib.pyplot as plt
from uncertainties import ufloat
from uncertainties.core import Variable, AffineScalarFunc
from dataclasses import dataclass
import scipy.constants as cst
from pathlib import Path

In [2]:
@dataclass
class PeakMeasurement:
    height: ufloat
    width: ufloat

In [3]:
def thermal_population(J, T, B=6.66733e9, n = 100):
    """calculate the thermal population of a given J sublevel
    Args:
        J (int): rotational level
        T (float): temperature [Kelvin]
        B (float, optional): rotational constant. Defaults to 6.66733e9.
        n (int, optional): number of rotational levels to normalize with. 
                            Defaults to 100.
    Returns:
        float: relative population in a rotational sublevel
    """
    c = 2*np.pi*cst.hbar*B/(cst.k*T)
    g = lambda J: 4*(2*J+1)
    a = lambda J: -c*J*(J+1)
    Z = np.sum([g(i)*np.exp(a(i)) for i in range(n)])
    return g(J)*np.exp(a(J))/Z

def J_levels(J):
    """calculate the number of hyperfine sublevels per J rotational level
    Args:
        J (int): rotational level
    Returns:
        int: number of levels
    """
    return 4*(2*J + 1)

In [4]:
def calculate_photons_sim(transition, df, T):
    J = int(transition[2])
    idx = np.where(df.transition == transition)[0][0]
    signal = df.signal[idx]
    ρ_thermal = thermal_population(J, T)
    indices = np.array(df.index)
    others = indices[indices != idx]
    # calculate photons in transition from other transitions
    # based on measured signals
    nphotons = []
    nphotons_sim = df["#photons_sim"].values
    levels = df.levels.values
    for idy in others:
        J_other = int(df.transition[idy][2])
        ρ_thermal_other = thermal_population(J_other, T)
        ratio_signal = signal/df.signal[idy]
        ratio_levels_sim = levels[idy]/J_levels(J_other)
        ratio_levels_tocalc = levels[idx]/J_levels(J)
        _ = ratio_signal * nphotons_sim[idy] * ratio_levels_sim * ρ_thermal_other / \
            ( ratio_levels_tocalc * ρ_thermal )
        nphotons.append(_)
    nphotons = np.array(nphotons)
    transitions = df.transition.values[others]
    return [*zip(transitions, nphotons)]

In [5]:
transitions = [
    centrex.transitions.LaserTransition("R1", F1=3/2, F=1),
    centrex.transitions.LaserTransition("R1", F1=3/2, F=2),
    centrex.transitions.LaserTransition("R1", F1=5/2, F=2),
    centrex.transitions.LaserTransition("R1", F1=5/2, F=3),
    centrex.transitions.LaserTransition("Q1", F1=1/2, F=0),
    centrex.transitions.LaserTransition("Q1", F1=1/2, F=1),
    centrex.transitions.LaserTransition("Q1", F1=3/2, F=1),
]

In [6]:
path = Path()

simulated_heights = {}
for p in path.glob("*.pkl"):
    print(p)
    with open(p, 'rb') as f:
        data = pickle.load(f)
        simulated_heights[str(p.stem)] = np.max(data['signal'])

Q(1) F1'=1_2 F'=0.pkl
Q(1) F1'=1_2 F'=1.pkl
Q(1) F1'=3_2 F'=1.pkl
R(1) F1'=3_2 F'=1.pkl
R(1) F1'=3_2 F'=2.pkl
R(1) F1'=5_2 F'=2.pkl
R(1) F1'=5_2 F'=3.pkl


In [7]:
# order the simulations in the same order as transitions
_ = []
for transition in transitions:
    _.append(simulated_heights[transition.get_string_os()])
simulated_heights = _
del _

In [8]:
measurements = [
    PeakMeasurement(ufloat(23.74, 0.76), ufloat(10.67, 0.55)),
    PeakMeasurement(ufloat(17.52, 0.47), ufloat(20.77, 1.02)),
    PeakMeasurement(ufloat(13.27, 0.54), ufloat(15.80, 1.11)),
    PeakMeasurement(ufloat(7.30, 0.40), ufloat(31.65, 3.52)),
    PeakMeasurement(ufloat(10.34, 0.52), ufloat(18.13, 1.61)),
    PeakMeasurement(ufloat(183.59, 2.98), ufloat(7.27, 0.18)),
    PeakMeasurement(ufloat(38.01, 0.96), ufloat(10.42, 0.42)),
]

In [9]:
# without vibrational, edit such that none exceed 100 photons cycled
branching = [
    0.7094209682686317,
    0.7118596758679805,
    0.5249999224639202,
    0.5235469186730339,
    0.99, #0.9999999938796454,
    0.99, #0.9998733406269951,
    0.8907056877229965
]

In [10]:
df = pd.DataFrame(
    {
        "transition": [t.get_string() for t in transitions],
        "levels": [t.n_ground for t in transitions],
        "#photons_branch": [round(1/(1-n),1) for n in branching],
        "#photons_sim": [round(s,1) for s in simulated_heights],
        "signal": [m.height for m in measurements],
    }
)
df

,transition,levels,#photons_branch,#photons_sim,signal
0,R(1) F1'=3/2 F'=1,12,3.4,2.8,23.7+/-0.8
1,R(1) F1'=3/2 F'=2,11,3.5,2.5,17.5+/-0.5
2,R(1) F1'=5/2 F'=2,11,2.1,1.9,13.3+/-0.5
3,R(1) F1'=5/2 F'=3,5,2.1,2.1,7.3+/-0.4
4,Q(1) F1'=1/2 F'=0,6,100.0,2.0,10.3+/-0.5
5,Q(1) F1'=1/2 F'=1,12,100.0,14.4,183.6+/-3.0
6,Q(1) F1'=3/2 F'=1,12,9.1,3.5,38.0+/-1.0


In [11]:
df_nphotons = pd.DataFrame()

T = 4 # Kelvin

for nphotons_sim, transition in zip(df["#photons_sim"], df.transition):
    _ = calculate_photons_sim(transition, df, T)
    header = f"Calculate #photons for {transition} from other transitions"
    print("-"*len(header))
    print(header)
    print(f"\t  #photons sim -> {nphotons_sim}")
    print("-"*len(header))
    for t, nphotons in _:
        print(f"{t} -> {nphotons} photons / molecule")
    dat = []
    for trans in transitions:
        found_transition = False
        for t, nphotons in _:
            if trans.get_string() == t:
                dat.append(nphotons)
                found_transition = True
                break
        if not found_transition:
            dat.append(np.nan)
        
    df_nphotons[transition] = dat
    
df_nphotons.index = [t.get_string() for t in transitions]

---------------------------------------------------------------
Calculate #photons for R(1) F1'=3/2 F'=1 from other transitions
	  #photons sim -> 2.8
---------------------------------------------------------------
R(1) F1'=3/2 F'=2 -> 3.11+/-0.13 photons / molecule
R(1) F1'=5/2 F'=2 -> 3.12+/-0.16 photons / molecule
R(1) F1'=5/2 F'=3 -> 2.85+/-0.18 photons / molecule
Q(1) F1'=1/2 F'=0 -> 2.30+/-0.14 photons / molecule
Q(1) F1'=1/2 F'=1 -> 1.86+/-0.07 photons / molecule
Q(1) F1'=3/2 F'=1 -> 2.19+/-0.09 photons / molecule
---------------------------------------------------------------
Calculate #photons for R(1) F1'=3/2 F'=2 from other transitions
	  #photons sim -> 2.5
---------------------------------------------------------------
R(1) F1'=3/2 F'=1 -> 2.25+/-0.09 photons / molecule
R(1) F1'=5/2 F'=2 -> 2.51+/-0.12 photons / molecule
R(1) F1'=5/2 F'=3 -> 2.29+/-0.14 photons / molecule
Q(1) F1'=1/2 F'=0 -> 1.85+/-0.11 photons / molecule
Q(1) F1'=1/2 F'=1 -> 1.50+/-0.05 photons / molecul

In [12]:
_ = pd.DataFrame()
for photons_sim, transition in zip(df["#photons_sim"], df.transition):
    _[transition] = [photons_sim]
_.index = ["#photons sim"]
df_nphotons = pd.concat([df_nphotons, _])

_ = pd.DataFrame()
for photons_branch, transition in zip(df["#photons_branch"], df.transition):
    _[transition] = [photons_branch]
_.index = ["#photons branching"]
df_nphotons = pd.concat([df_nphotons, _])

In [13]:
df_nphotons

,R(1) F1'=3/2 F'=1,R(1) F1'=3/2 F'=2,R(1) F1'=5/2 F'=2,R(1) F1'=5/2 F'=3,Q(1) F1'=1/2 F'=0,Q(1) F1'=1/2 F'=1,Q(1) F1'=3/2 F'=1
R(1) F1'=3/2 F'=1,NaN,2.25+/-0.09,1.71+/-0.09,2.07+/-0.13,2.44+/-0.15,21.7+/-0.8,4.48+/-0.18
R(1) F1'=3/2 F'=2,3.11+/-0.13,NaN,1.89+/-0.09,2.29+/-0.14,2.71+/-0.15,24.0+/-0.8,4.97+/-0.18
R(1) F1'=5/2 F'=2,3.12+/-0.16,2.51+/-0.12,NaN,2.30+/-0.16,2.71+/-0.18,24.1+/-1.1,4.99+/-0.24
R(1) F1'=5/2 F'=3,2.85+/-0.18,2.29+/-0.14,1.74+/-0.12,NaN,2.48+/-0.18,22.0+/-1.3,4.56+/-0.27
Q(1) F1'=1/2 F'=0,2.30+/-0.14,1.85+/-0.11,1.40+/-0.09,1.69+/-0.13,NaN,17.8+/-0.9,3.68+/-0.21
Q(1) F1'=1/2 F'=1,1.86+/-0.07,1.50+/-0.05,1.14+/-0.05,1.37+/-0.08,1.62+/-0.09,NaN,2.98+/-0.09
Q(1) F1'=3/2 F'=1,2.19+/-0.09,1.76+/-0.06,1.33+/-0.06,1.61+/-0.10,1.90+/-0.11,16.9+/-0.5,NaN
#photons sim,2.8,2.5,1.9,2.1,2.0,14.4,3.5
#photons branching,3.4,3.5,2.1,2.1,100.0,100.0,9.1


In [15]:
def style_within_sigma(v, props = "", **kwargs):
    value = kwargs.get("value", np.nan)
    nsigma = kwargs.get("nsigma", 2)
    if isinstance(v, (Variable, AffineScalarFunc)):
        return props if (np.abs(v.n - value) < nsigma*v.s) else None
    if np.isnan(v):
        return None
    
def style_outside_sigma(v, props = "", **kwargs):
    value = kwargs.get("value", np.nan)
    nsigma = kwargs.get("nsigma", 5)
    if isinstance(v, (Variable, AffineScalarFunc)):
        return props if (np.abs(v.n - value) > nsigma*v.s) else None
    if np.isnan(v):
        return None


In [16]:
nsigma_good = 2

index_string = "["
for index in df_nphotons.index[:-2]:
    index_string += '"' + str(index) + '",'
index_string = index_string.strip(",") + "]"
    
# TODO: find a better method without eval
cmd_string = "df_nphotons.style"
for transition in df_nphotons.columns:
    value = df_nphotons[transition]["#photons sim"]
    subset_string = '["' +  str(transition) + '"]'
    cmd_string += (
        f".applymap("
        f"style_within_sigma,"
        f"props = 'color:white;background-color:green;opacity: 50%;',"
        f"value = {value},"
        f"nsigma = {nsigma_good},"
        f'subset = ({index_string}, {subset_string})'
        ')'
    )
# for transition in df_nphotons.columns:
#     value = df_nphotons[transition]["#photons sim"]
#     cmd_string += (
#         f".applymap("
#         f"style_outside_sigma,"
#         f"props = 'color:white;background-color:red;opacity: 50%;',"
#         f"value = {value},"
#         'subset = ["'+str(transition)+ '"])'
#     )
df_nphotons_style = eval(cmd_string)
df_nphotons_style

,R(1) F1'=3/2 F'=1,R(1) F1'=3/2 F'=2,R(1) F1'=5/2 F'=2,R(1) F1'=5/2 F'=3,Q(1) F1'=1/2 F'=0,Q(1) F1'=1/2 F'=1,Q(1) F1'=3/2 F'=1
R(1) F1'=3/2 F'=1,nan,2.25+/-0.09,1.71+/-0.09,2.07+/-0.13,2.44+/-0.15,21.7+/-0.8,4.48+/-0.18
R(1) F1'=3/2 F'=2,3.11+/-0.13,nan,1.89+/-0.09,2.29+/-0.14,2.71+/-0.15,24.0+/-0.8,4.97+/-0.18
R(1) F1'=5/2 F'=2,3.12+/-0.16,2.51+/-0.12,nan,2.30+/-0.16,2.71+/-0.18,24.1+/-1.1,4.99+/-0.24
R(1) F1'=5/2 F'=3,2.85+/-0.18,2.29+/-0.14,1.74+/-0.12,nan,2.48+/-0.18,22.0+/-1.3,4.56+/-0.27
Q(1) F1'=1/2 F'=0,2.30+/-0.14,1.85+/-0.11,1.40+/-0.09,1.69+/-0.13,nan,17.8+/-0.9,3.68+/-0.21
Q(1) F1'=1/2 F'=1,1.86+/-0.07,1.50+/-0.05,1.14+/-0.05,1.37+/-0.08,1.62+/-0.09,nan,2.98+/-0.09
Q(1) F1'=3/2 F'=1,2.19+/-0.09,1.76+/-0.06,1.33+/-0.06,1.61+/-0.10,1.90+/-0.11,16.9+/-0.5,nan
#photons sim,2.800000,2.500000,1.900000,2.100000,2.000000,14.400000,3.500000
#photons branching,3.400000,3.500000,2.100000,2.100000,100.000000,100.000000,9.100000
